In [3]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.3 MB/s

In [1]:
!pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7922 sha256=7ea8c4315d843cab8a2171b70fedf33ff5ac6d479c34e2f996a364e16ebcbc37
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [4]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
from pytimekr import pytimekr

감귤 데이터만 따로 만들어보았다.

In [8]:
tg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DACON/제주 특산물 가격예측/open (3)/open/TG_data.csv')[['ID','timestamp','corporation', 'price(원/kg)' ,'supply(kg)']]
tg['item_id'] = tg.ID.str[0:6]

In [9]:
tg

,ID,timestamp,corporation,price(원/kg),supply(kg),item_id
0,TG_A_J_20190101,2019-01-01,A,0.0,0.0,TG_A_J
1,TG_A_J_20190102,2019-01-02,A,0.0,0.0,TG_A_J
2,TG_A_J_20190103,2019-01-03,A,1728.0,60601.0,TG_A_J
3,TG_A_J_20190104,2019-01-04,A,1408.0,25000.0,TG_A_J
4,TG_A_J_20190105,2019-01-05,A,1250.0,32352.0,TG_A_J
...,...,...,...,...,...,...
15225,TG_E_S_20230227,2023-02-27,E,3418.0,24204.0,TG_E_S
15226,TG_E_S_20230228,2023-02-28,E,3141.0,13587.0,TG_E_S
15227,TG_E_S_20230301,2023-03-01,E,4235.0,16187.0,TG_E_S
15228,TG_E_S_20230302,2023-03-02,E,3960.0,17830.0,TG_E_S


In [11]:
import datetime
tg['timestamp'] = pd.to_datetime(tg['timestamp'])

In [29]:
year_2019 = pytimekr.holidays(year=2019)
year_2020 = pytimekr.holidays(year=2020)
year_2021 = pytimekr.holidays(year=2021)
year_2022 = pytimekr.holidays(year=2022)
year_2023 = pytimekr.holidays(year=2023)



def holidays(x):
    if x.weekday() in range(5,8):
        return 1
    if x.year == 2019  and x in year_2019 :
        return 1
    elif x.year == 2020 and x in year_2020:
        return 1
    elif x.year == 2021 and x in year_2021 :
        return 1
    elif x.year == 2022 and x in year_2022 :
        return 1
    elif x.year == 2023 and x in year_2023:
        return 1
    else:
        return 0

import warnings
warnings.filterwarnings('ignore')
tg['holiday'] = tg['timestamp'].apply(holidays)
tg.head(7)

,ID,timestamp,corporation,price(원/kg),supply(kg),item_id,holiday,std_supply
0,TG_A_J_20190101,2019-01-01,A,0.0,0.0,TG_A_J,1,-0.558499
1,TG_A_J_20190102,2019-01-02,A,0.0,0.0,TG_A_J,0,-0.558499
2,TG_A_J_20190103,2019-01-03,A,1728.0,60601.0,TG_A_J,0,1.826129
3,TG_A_J_20190104,2019-01-04,A,1408.0,25000.0,TG_A_J,0,0.425242
4,TG_A_J_20190105,2019-01-05,A,1250.0,32352.0,TG_A_J,1,0.714541
5,TG_A_J_20190106,2019-01-06,A,0.0,0.0,TG_A_J,1,-0.558499
6,TG_A_J_20190107,2019-01-07,A,1474.0,44995.0,TG_A_J,0,1.212039


In [30]:
# Modify the 'holiday' column based on the conditions provided
tg['holiday'] = tg.apply(lambda row: 1 if (row['price(원/kg)'] == 0 and row['supply(kg)'] == 0) else 0, axis=1)

# Display the first 7 rows to verify the changes
tg.head(20)

,ID,timestamp,corporation,price(원/kg),supply(kg),item_id,holiday,std_supply
0,TG_A_J_20190101,2019-01-01,A,0.0,0.0,TG_A_J,1,-0.558499
1,TG_A_J_20190102,2019-01-02,A,0.0,0.0,TG_A_J,1,-0.558499
2,TG_A_J_20190103,2019-01-03,A,1728.0,60601.0,TG_A_J,0,1.826129
3,TG_A_J_20190104,2019-01-04,A,1408.0,25000.0,TG_A_J,0,0.425242
4,TG_A_J_20190105,2019-01-05,A,1250.0,32352.0,TG_A_J,0,0.714541
5,TG_A_J_20190106,2019-01-06,A,0.0,0.0,TG_A_J,1,-0.558499
6,TG_A_J_20190107,2019-01-07,A,1474.0,44995.0,TG_A_J,0,1.212039
7,TG_A_J_20190108,2019-01-08,A,1326.0,26975.0,TG_A_J,0,0.502958
8,TG_A_J_20190109,2019-01-09,A,1428.0,29265.0,TG_A_J,0,0.593068
9,TG_A_J_20190110,2019-01-10,A,1433.0,21226.0,TG_A_J,0,0.276737


일요일과 공휴일 당일에는 유통이 일어나지 않아서 이를 구분할 필요성이 있다고 생각해 holiday 열을 추가해주었다.

supply 표준화 전처리& corporation 원핫 인코딩& 멀티 인덱스

In [31]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

tg['std_supply'] = std_scaler.fit_transform(tg['supply(kg)'].values.reshape(-1,1))

In [34]:
data = TimeSeriesDataFrame(tg.drop(columns=['ID'])) # 멀티 인덱스
data = pd.get_dummies(data, columns=['corporation','holiday'])

tg_df = data.drop('supply(kg)', axis=1)

In [35]:
tg_df

price(원/kg)  std_supply  corporation_A  corporation_B  \
item_id timestamp                                                           
TG_A_J  2019-01-01          0.0   -0.558499              1              0   
        2019-01-02          0.0   -0.558499              1              0   
        2019-01-03       1728.0    1.826129              1              0   
        2019-01-04       1408.0    0.425242              1              0   
        2019-01-05       1250.0    0.714541              1              0   
...                         ...         ...            ...            ...   
TG_E_S  2023-02-27       3418.0    0.393920              0              0   
        2023-02-28       3141.0   -0.023855              0              0   
        2023-03-01       4235.0    0.078454              0              0   
        2023-03-02       3960.0    0.143105              0              0   
        2023-03-03       3791.0    0.470770              0              0   

                    corporation_C  corporation_D  corporation_E  holiday_0  \
item_id timestamp                                                            
TG_A_J  2019-01-01              0              0              0          0   
        2019-01-02              0              0              0          0   
        2019-01-03              0              0              0          1   
        2019-01-04              0              0              0          1   
        2019-01-05              0              0              0          1   
...                           ...            ...            ...        ...   
TG_E_S  2023-02-27              0              0              1          1   
        2023-02-28              0              0              1          1   
        2023-03-01              0              0              1          1   
        2023-03-02              0              0              1          1   
        2023-03-03              0              0              1          1   

                    holiday_1  
item_id timestamp              
TG_A_J  2019-01-01          1  
        2019-01-02          1  
        2019-01-03          0  
        2019-01-04          0  
        2019-01-05          0  
...                       ...  
TG_E_S  2023-02-27          0  
        2023-02-28          0  
        2023-03-01          0  
        2023-03-02          0  
        2023-03-03          0  

[15230 rows x 9 columns]

In [36]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",

)

# seed 고정
predictor.fit( tg_df, random_seed=42) # num_val_windows 수 변경

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 15230 rows, 10 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231116_144111/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['std_supply', 

In [37]:
# Retrain model on all of the data (training + validation).
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	1.72    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 1.77 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [40]:
pred = predictor.predict(tg_df, random_seed=42,)
pred.head(27)


INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3354.411573  1922.626021  2479.234264  2804.143098   
        2023-03-05   563.680452  -415.291817   -78.733413   163.480165   
        2023-03-06  3506.355442  1451.251567  2302.195333  2770.262025   
        2023-03-07  3620.936978  1406.705228  2323.530602  2795.055789   
        2023-03-08  3625.617688  1473.524759  2247.383517  2836.831249   
        2023-03-09  3489.042005  1055.045473  1953.550453  2503.372692   
        2023-03-10  3625.740923   604.069953  1881.615615  2582.541373   
        2023-03-11  3399.096961   464.269317  1590.851077  2408.791370   
        2023-03-12   572.965551 -1501.884384  -789.719903  -276.079602   
        2023-03-13  3378.284004   105.780402  1380.047697  2161.676701   
        2023-03-14  3518.586292    84.132182  1246.577889  2096.616913   
        2023-03-15  3497.594181  -300.529706  1237.558387  2127.102415   
        2023-03-16  3796.913196    72.933656  1494.577114  2376.362314   
        2023-03-17  3956.730536  -130.822402  1345.565949  2421.961203   
        2023-03-18  3719.228179  -463.238093  1015.208461  2099.336910   
        2023-03-19   582.585385 -2185.231067 -1235.763487  -550.807704   
        2023-03-20  3784.607829  -582.497087  1012.368154  2116.372397   
        2023-03-21  4067.428962  -960.794927  1064.606643  2141.693720   
        2023-03-22  4104.378339  -763.562039  1070.405636  2185.354815   
        2023-03-23  3731.870542 -1372.953989   657.217630  1880.046686   
        2023-03-24  3862.878234 -1410.360659   597.211085  2154.265382   
        2023-03-25  3622.682375 -1982.753685   337.167807  1730.076850   
        2023-03-26   587.397034 -2732.399382 -1590.639180  -768.961728   
        2023-03-27  3359.590861 -2275.092482   -96.679719  1301.682898   
        2023-03-28  3121.828449 -3120.903971  -724.818165   997.128406   
        2023-03-29  3526.829625 -2818.379025  -580.999720  1309.002018   
        2023-03-30  3371.871730 -2928.854315  -494.778152  1189.536180   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3102.983005  3380.489210  3609.408257  3881.203975   
        2023-03-05   370.476337   564.010150   757.369923   964.478817   
        2023-03-06  3140.919062  3519.857151  3898.872388  4272.793097   
        2023-03-07  3217.651618  3645.718960  4066.302171  4480.435355   
        2023-03-08  3331.291450  3745.770520  4167.902422  4655.850173   
        2023-03-09  3064.665695  3561.221205  4016.432720  4562.792618   
        2023-03-10  3180.141179  3778.243365  4286.221771  4843.129439   
        2023-03-11  2986.878105  3532.817909  4055.007956  4611.937953   
        2023-03-12   162.488738   572.611304   982.562608  1421.371176   
        2023-03-13  2844.534795  3413.116768  4021.201670  4710.279490   
        2023-03-14  2829.621689  3442.312122  4101.863640  4754.321052   
        2023-03-15  2885.892563  3547.032169  4190.219382  4982.727548   
        2023-03-16  3108.515339  3730.910267  4435.781757  5183.631315   
        2023-03-17  3221.455965  3897.448798  4604.553447  5483.490592   
        2023-03-18  3019.215020  3769.067291  4516.926397  5353.624575   
        2023-03-19    34.272899   581.007077  1127.690133  1712.464355   
        2023-03-20  3023.606211  3771.341472  4451.816722  5283.159911   
        2023-03-21  3145.567841  3962.838386  4800.189985  5762.723286   
        2023-03-22  3169.040572  3980.037274  4904.896037  5866.773933   
        2023-03-23  2904.816943  3886.247251  4908.751778  5897.632142   
        2023-03-24  3141.118583  4147.452209  5038.154732  6059.806028   
        2023-03-25  2647.904799  3650.571535  4719.444325  5676.926571   
        2023-03-26   -66.975383   588.757079  1244.174932  1946.336457   
        2023-03-27  2381.27598

In [44]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DACON/제주 특산물 가격예측/open (3)/open/tg_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 700.0, 'answer'] = 0.0

submission

,ID,answer
0,TG_A_J_20230304,3354.411573
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3506.355442
3,TG_A_J_20230307,3620.936978
4,TG_A_J_20230308,3625.617688
...,...,...
275,TG_E_S_20230327,4157.796379
276,TG_E_S_20230328,4041.608576
277,TG_E_S_20230329,4264.650503
278,TG_E_S_20230330,3918.108374


In [45]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/DACON/제주 특산물 가격예측/open (3)/open/tg_submission.csv', index=False)
submission

,ID,answer
0,TG_A_J_20230304,3354.411573
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3506.355442
3,TG_A_J_20230307,3620.936978
4,TG_A_J_20230308,3625.617688
...,...,...
275,TG_E_S_20230327,4157.796379
276,TG_E_S_20230328,4041.608576
277,TG_E_S_20230329,4264.650503
278,TG_E_S_20230330,3918.108374


In [84]:
no_tg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DACON/제주 특산물 가격예측/open (3)/open/no_TG (2).csv')[['ID','timestamp','corporation', 'location', 'price(원/kg)' ,'supply(kg)']]

no_tg

,ID,timestamp,corporation,location,price(원/kg),supply(kg)
0,CR_A_J_20190101,2019-01-01,A,J,0,0.0
1,CR_A_J_20190102,2019-01-02,A,J,0,0.0
2,CR_A_J_20190103,2019-01-03,A,J,0,0.0
3,CR_A_J_20190104,2019-01-04,A,J,1141,10240.0
4,CR_A_J_20190105,2019-01-05,A,J,1133,8680.0
...,...,...,...,...,...,...
44162,RD_F_J_20230227,2023-02-27,F,J,468,452440.0
44163,RD_F_J_20230228,2023-02-28,F,J,531,421980.0
44164,RD_F_J_20230301,2023-03-01,F,J,574,382980.0
44165,RD_F_J_20230302,2023-03-02,F,J,523,477220.0


In [85]:
no_tg['timestamp'] = pd.to_datetime(no_tg['timestamp'])

year_2019 = pytimekr.holidays(year=2019)
year_2020 = pytimekr.holidays(year=2020)
year_2021 = pytimekr.holidays(year=2021)
year_2022 = pytimekr.holidays(year=2022)
year_2023 = pytimekr.holidays(year=2023)



def holidays(x):
    if x.weekday() in range(5,8):
        return 1
    if x.year == 2019  and x in year_2019 :
        return 1
    elif x.year == 2020 and x in year_2020:
        return 1
    elif x.year == 2021 and x in year_2021 :
        return 1
    elif x.year == 2022 and x in year_2022 :
        return 1
    elif x.year == 2023 and x in year_2023:
        return 1
    else:
        return 0

import warnings
warnings.filterwarnings('ignore')
no_tg['holiday'] = no_tg['timestamp'].apply(holidays)
no_tg.head(7)

# Modify the 'holiday' column based on the conditions provided
no_tg['holiday'] = no_tg.apply(lambda row: 1 if (row['price(원/kg)'] == 0 and row['supply(kg)'] == 0) else 0, axis=1)

# Display the first 7 rows to verify the changes
no_tg.head(100)

,ID,timestamp,corporation,location,price(원/kg),supply(kg),holiday
0,CR_A_J_20190101,2019-01-01,A,J,0,0.0,1
1,CR_A_J_20190102,2019-01-02,A,J,0,0.0,1
2,CR_A_J_20190103,2019-01-03,A,J,0,0.0,1
3,CR_A_J_20190104,2019-01-04,A,J,1141,10240.0,0
4,CR_A_J_20190105,2019-01-05,A,J,1133,8680.0,0
...,...,...,...,...,...,...,...
95,CR_A_J_20190406,2019-04-06,A,J,0,0.0,1
96,CR_A_J_20190407,2019-04-07,A,J,0,0.0,1
97,CR_A_J_20190408,2019-04-08,A,J,961,10080.0,0
98,CR_A_J_20190409,2019-04-09,A,J,1117,18000.0,0


In [86]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

no_tg['std_supply'] = std_scaler.fit_transform(no_tg['supply(kg)'].values.reshape(-1,1))

In [87]:
no_tg

,ID,timestamp,corporation,location,price(원/kg),supply(kg),holiday,std_supply
0,CR_A_J_20190101,2019-01-01,A,J,0,0.0,1,-0.189059
1,CR_A_J_20190102,2019-01-02,A,J,0,0.0,1,-0.189059
2,CR_A_J_20190103,2019-01-03,A,J,0,0.0,1,-0.189059
3,CR_A_J_20190104,2019-01-04,A,J,1141,10240.0,0,-0.014677
4,CR_A_J_20190105,2019-01-05,A,J,1133,8680.0,0,-0.041243
...,...,...,...,...,...,...,...,...
44162,RD_F_J_20230227,2023-02-27,F,J,468,452440.0,0,7.515780
44163,RD_F_J_20230228,2023-02-28,F,J,531,421980.0,0,6.997060
44164,RD_F_J_20230301,2023-03-01,F,J,574,382980.0,0,6.332909
44165,RD_F_J_20230302,2023-03-02,F,J,523,477220.0,0,7.937771


In [90]:
no_tg['timestamp'] = pd.to_datetime(no_tg['timestamp'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44167 entries, 0 to 44166
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           44167 non-null  object        
 1   timestamp    44167 non-null  datetime64[ns]
 2   corporation  44167 non-null  object        
 3   location     44167 non-null  object        
 4   price(원/kg)  44167 non-null  int64         
 5   supply(kg)   44167 non-null  float64       
 6   holiday      44167 non-null  int64         
 7   std_supply   44167 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 2.7+ MB


In [91]:
no_tg['item_id'] = no_tg.ID.str[0:6]

data = TimeSeriesDataFrame(no_tg.drop(columns=['ID'])) # 멀티 인덱스
train_data = pd.get_dummies(data, columns=['corporation', 'location','holiday'])

In [92]:
train_data

price(원/kg)  supply(kg)  std_supply  corporation_A  \
item_id timestamp                                                        
CR_A_J  2019-01-01            0         0.0   -0.189059              1   
        2019-01-02            0         0.0   -0.189059              1   
        2019-01-03            0         0.0   -0.189059              1   
        2019-01-04         1141     10240.0   -0.014677              1   
        2019-01-05         1133      8680.0   -0.041243              1   
...                         ...         ...         ...            ...   
RD_F_J  2023-02-27          468    452440.0    7.515780              0   
        2023-02-28          531    421980.0    6.997060              0   
        2023-03-01          574    382980.0    6.332909              0   
        2023-03-02          523    477220.0    7.937771              0   
        2023-03-03          529    427520.0    7.091404              0   

                    corporation_B  corporation_C  corporation_D  \
item_id timestamp                                                 
CR_A_J  2019-01-01              0              0              0   
        2019-01-02              0              0              0   
        2019-01-03              0              0              0   
        2019-01-04              0              0              0   
        2019-01-05              0              0              0   
...                           ...            ...            ...   
RD_F_J  2023-02-27              0              0              0   
        2023-02-28              0              0              0   
        2023-03-01              0              0              0   
        2023-03-02              0              0              0   
        2023-03-03              0              0              0   

                    corporation_E  corporation_F  location_J  location_S  \
item_id timestamp                                                          
CR_A_J  2019-01-01              0              0           1           0   
        2019-01-02              0              0           1           0   
        2019-01-03              0              0           1           0   
        2019-01-04              0              0           1           0   
        2019-01-05              0              0           1           0   
...                           ...            ...         ...         ...   
RD_F_J  2023-02-27              0              1           1           0   
        2023-02-28              0              1           1           0   
        2023-03-01              0              1           1           0   
        2023-03-02              0              1           1           0   
        2023-03-03              0              1           1           0   

                    holiday_0  holiday_1  
item_id timestamp                         
CR_A_J  2019-01-01          0          1  
        2019-01-02          0          1  
        2019-01-03          0          1  
        2019-01-04          1          0  
        2019-01-05          1          0  
...                       ...        ...  
RD_F_J  2023-02-27          1          0  
        2023-02-28          1          0  
        2023-03-01          1          0  
        2023-03-02          1          0  
        2023-03-03          1          0  

[44167 rows x 13 columns]

In [93]:
df = train_data.drop('supply(kg)', axis=1)

In [94]:
df

price(원/kg)  std_supply  corporation_A  corporation_B  \
item_id timestamp                                                           
CR_A_J  2019-01-01            0   -0.189059              1              0   
        2019-01-02            0   -0.189059              1              0   
        2019-01-03            0   -0.189059              1              0   
        2019-01-04         1141   -0.014677              1              0   
        2019-01-05         1133   -0.041243              1              0   
...                         ...         ...            ...            ...   
RD_F_J  2023-02-27          468    7.515780              0              0   
        2023-02-28          531    6.997060              0              0   
        2023-03-01          574    6.332909              0              0   
        2023-03-02          523    7.937771              0              0   
        2023-03-03          529    7.091404              0              0   

                    corporation_C  corporation_D  corporation_E  \
item_id timestamp                                                 
CR_A_J  2019-01-01              0              0              0   
        2019-01-02              0              0              0   
        2019-01-03              0              0              0   
        2019-01-04              0              0              0   
        2019-01-05              0              0              0   
...                           ...            ...            ...   
RD_F_J  2023-02-27              0              0              0   
        2023-02-28              0              0              0   
        2023-03-01              0              0              0   
        2023-03-02              0              0              0   
        2023-03-03              0              0              0   

                    corporation_F  location_J  location_S  holiday_0  \
item_id timestamp                                                      
CR_A_J  2019-01-01              0           1           0          0   
        2019-01-02              0           1           0          0   
        2019-01-03              0           1           0          0   
        2019-01-04              0           1           0          1   
        2019-01-05              0           1           0          1   
...                           ...         ...         ...        ...   
RD_F_J  2023-02-27              1           1           0          1   
        2023-02-28              1           1           0          1   
        2023-03-01              1           1           0          1   
        2023-03-02              1           1           0          1   
        2023-03-03              1           1           0          1   

                    holiday_1  
item_id timestamp              
CR_A_J  2019-01-01          1  
        2019-01-02          1  
        2019-01-03          1  
        2019-01-04          0  
        2019-01-05          0  
...                       ...  
RD_F_J  2023-02-27          0  
        2023-02-28          0  
        2023-03-01          0  
        2023-03-02          0  
        2023-03-03          0  

[44167 rows x 12 columns]

In [96]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",

)

# seed 고정
predictor.fit( df, random_seed=42) # num_val_windows 수 변경

No path specified. Models will be saved in: "AutogluonModels/ag-20231116_154228/"
================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 44167 rows, 29 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231116_154228/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains foll

In [97]:
# Retrain model on all of the data (training + validation).
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	2.01    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 2.06 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [98]:
pred = predictor.predict(df, random_seed=42,)
pred.head(27)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


mean          0.1          0.2          0.3  \
item_id timestamp                                                        
CR_A_J  2023-03-04  1922.669871  1318.984275  1551.538614  1697.204698   
        2023-03-05   376.567904  -206.195411    -0.294794   143.140990   
        2023-03-06  2200.494163  1484.578696  1735.081030  1900.523990   
        2023-03-07  2278.556809  1549.558635  1807.485436  2000.281846   
        2023-03-08  2252.821107  1429.024417  1733.656979  1939.934997   
        2023-03-09  2167.055335  1331.185279  1630.525845  1839.206966   
        2023-03-10  2719.784267  1830.480354  2153.095179  2366.075916   
        2023-03-11  1706.586680   718.589949  1061.760470  1320.557422   
        2023-03-12   373.169130  -436.927638  -156.938907    40.168674   
        2023-03-13  2108.067528  1153.247825  1482.918383  1708.258660   
        2023-03-14  2184.655502  1166.773658  1553.830036  1792.464773   
        2023-03-15  2258.125977  1239.794707  1596.527544  1838.754266   
        2023-03-16  2072.765235   998.541144  1369.651972  1647.699927   
        2023-03-17  2671.828334  1507.164956  1960.820965  2246.703102   
        2023-03-18  1590.577800   472.312703   835.341768  1124.326999   
        2023-03-19   376.044698  -592.662828  -257.488763   -19.074073   
        2023-03-20  2050.738609   898.384512  1308.033047  1603.842762   
        2023-03-21  2191.082871   966.364956  1425.985248  1727.190452   
        2023-03-22  2192.971375   942.270651  1429.767049  1725.487108   
        2023-03-23  2104.689124   851.501901  1311.157252  1631.849174   
        2023-03-24  2636.121669  1302.540312  1824.246602  2135.385004   
        2023-03-25  1716.857707   353.090128   870.281249  1210.717552   
        2023-03-26   371.478852  -729.373197  -353.134175   -76.507361   
        2023-03-27  2037.841087   699.770287  1254.262723  1568.166620   
        2023-03-28  2146.446884   709.282742  1331.075483  1669.625921   
        2023-03-29  2182.682374   704.544504  1294.838057  1656.869190   
        2023-03-30  2098.108313   625.358420  1203.076449  1560.231551   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
CR_A_J  2023-03-04  1821.680384  1930.852367  2042.556819  2158.661473   
        2023-03-05   261.724267   373.294470   487.208302   607.096474   
        2023-03-06  2052.693060  2198.033226  2336.461648  2490.170147   
        2023-03-07  2147.296215  2287.008225  2431.037739  2584.390060   
        2023-03-08  2099.052928  2250.819581  2413.138785  2573.372499   
        2023-03-09  2007.601717  2175.607398  2342.837017  2514.563799   
        2023-03-10  2565.435427  2735.988674  2904.784230  3083.476443   
        2023-03-11  1518.508393  1699.497600  1888.502645  2097.610507   
        2023-03-12   209.091972   367.487543   526.144908   694.434580   
        2023-03-13  1918.287449  2103.178917  2290.961925  2496.348136   
        2023-03-14  1988.325479  2185.371506  2382.626700  2578.320109   
        2023-03-15  2043.100324  2240.576356  2450.194599  2655.208138   
        2023-03-16  1873.164503  2083.457068  2302.131225  2521.183753   
        2023-03-17  2488.352745  2691.226222  2915.271331  3135.389784   
        2023-03-18  1367.346995  1587.858440  1794.878276  2036.621966   
        2023-03-19   184.532027   374.943716   565.305911   767.345301   
        2023-03-20  1848.842930  2059.870384  2289.484203  2525.124263   
        2023-03-21  1968.535006  2198.437729  2425.928464  2657.472237   
        2023-03-22  1981.251663  2212.458619  2440.956224  2689.025832   
        2023-03-23  1880.896359  2115.767188  2354.603288  2611.239920   
        2023-03-24  2421.646664  2676.651637  2929.401850  3186.764523   
        2023-03-25  1480.855735  1727.946208  1980.291137  2255.363463   
        2023-03-26   152.564529   367.044542   579.426532   811.961881   
        2023-03-27  1823.46162

In [102]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DACON/제주 특산물 가격예측/open (3)/open/not_tg_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0


In [103]:
submission

,ID,answer
0,CR_A_J_20230304,1922.669871
1,CR_A_J_20230305,376.567904
2,CR_A_J_20230306,2200.494163
3,CR_A_J_20230307,2278.556809
4,CR_A_J_20230308,2252.821107
...,...,...
807,RD_F_J_20230327,486.564534
808,RD_F_J_20230328,509.336711
809,RD_F_J_20230329,536.641190
810,RD_F_J_20230330,511.450228


In [104]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/DACON/제주 특산물 가격예측/open (3)/open/not_tg_submission.csv', index=False)
submission

,ID,answer
0,CR_A_J_20230304,1922.669871
1,CR_A_J_20230305,376.567904
2,CR_A_J_20230306,2200.494163
3,CR_A_J_20230307,2278.556809
4,CR_A_J_20230308,2252.821107
...,...,...
807,RD_F_J_20230327,486.564534
808,RD_F_J_20230328,509.336711
809,RD_F_J_20230329,536.641190
810,RD_F_J_20230330,511.450228
